In [1]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from tf_lab.fundamentals.utils import set_visible_GPUs, reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec


import tf_lab.autopredictors.scripts.virt_scan_data as vscan

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, points_extension


from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch
        
from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs, accuracy_of_completion, \
                                           coverage_of_completion
                                                  

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [9]:
incomplete_n_samples = 2048
complete_n_samples = 4096

class_type = raw_input('Give me the class type.\n').lower()
class_syn_id = shape_net_category_to_synth_id()[class_type]

GPU = 3
exp_counter = '8'
loss = 'emd'
do_training = True
first_time_running = True
load_model = False
do_test = False
do_plotting = False
training_epochs = 1000

experiment_name = exp_counter + '_' + class_type + '_' + str(incomplete_n_samples) + '_' + str(complete_n_samples) + 'pts_' + loss
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
complete_pclouds_path = osp.join(top_data_dir, 'ShapeNetPointClouds/from_manifold_meshes/', \
                             str(complete_n_samples), class_syn_id)
    
n_input = [incomplete_n_samples, 3]
n_output = [complete_n_samples, 3] 

train_dir = osp.join(top_data_dir, 'OUT/models/incomplete_pclouds/paper_vanilla_vscan')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)

Give me the class type.
cabinet


'/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/8_cabinet_2048_4096pts_emd'

In [10]:
seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)

In [11]:
cmpl_file_names = pio.load_filenames_of_input_data(complete_pclouds_path, points_extension)
cmpl_pclouds, cmpl_model_names, syn_ids = pio.load_crude_point_clouds(file_names=cmpl_file_names, n_threads=25)
assert(len(np.unique(syn_ids)) == 1)
print '%d files containing complete point clouds  were found.' % (len(cmpl_pclouds), )

1572 files containing complete point clouds  were found.


In [12]:
v_scan_top_dir = osp.join(top_data_dir, 'From_Matthias/shapenet_dim32_sdf_pc', class_syn_id)
if do_training:    
    train_id_file = osp.join(top_data_dir, 'From_Matthias/train_test_models/train_models_with_scan_id.txt')
    
    train_data = vscan.load_single_class_incomplete_dataset(v_scan_top_dir, train_id_file, class_syn_id,
                                                      cmpl_pclouds, cmpl_model_names, n_threads=25,
                                                      n_samples=incomplete_n_samples)
    
    train_data, val_data = vscan.make_validation_from_train_data(train_data, 0.10)

7800 incomplete point clouds were loaded.


In [13]:
if do_test:
    test_id_file = osp.join(top_data_dir, 'From_Matthias/train_test_models/test_models_with_scan_id.txt')
    test_data = vscan.load_single_class_incomplete_dataset(v_scan_top_dir, test_id_file, class_syn_id, 
                                                      cmpl_pclouds, cmpl_model_names, n_threads=25,
                                                      n_samples=incomplete_n_samples, search_pattern=search_pattern)

In [14]:
if load_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
else:
    decoder_args = {'layer_sizes': [1024, np.prod(n_output)],
                    'non_linearity': tf.nn.relu
                   }
    
    conf = Conf(
                n_input = n_input,
                n_output = n_output,
                denoising = True,
                training_epochs = training_epochs,
                batch_size = 50,
                loss = loss,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 5,
                learning_rate = 0.0005,
                gauss_augment = {'mu': 0, 'sigma': 0.02},
                encoder = enc_dec.encoder_with_convs_and_symmetry,                
                decoder = enc_dec.decoder_with_fc_only,
                decoder_args = decoder_args
               )
    
    conf.consistent_io = False
    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))

In [15]:
reset_tf_graph()
set_visible_GPUs([GPU])
ae = PointNetAutoEncoder(experiment_name, conf)

if do_training:
    ae.train(train_data, conf)

if load_model:
    saved_epochs = read_saved_epochs(conf.train_dir)
    ae.restore_model(train_dir, saved_epochs[-1], verbose=True)

('Epoch:', '0001', 'training time (minutes)=', '1.3076', 'loss=', '308.976298632')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/8_cabinet_2048_4096pts_emd/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.


INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/8_cabinet_2048_4096pts_emd/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.


('Epoch:', '0002', 'training time (minutes)=', '1.3273', 'loss=', '245.728862544')
('Epoch:', '0003', 'training time (minutes)=', '1.3356', 'loss=', '238.248026167')
('Epoch:', '0004', 'training time (minutes)=', '1.3311', 'loss=', '233.393596649')
('Epoch:', '0005', 'training time (minutes)=', '1.3261', 'loss=', '229.423761640')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/8_cabinet_2048_4096pts_emd/models.ckpt-5 is not in all_model_checkpoint_paths. Manually adding it.


INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/8_cabinet_2048_4096pts_emd/models.ckpt-5 is not in all_model_checkpoint_paths. Manually adding it.


('Epoch:', '0006', 'training time (minutes)=', '1.3286', 'loss=', '229.905786787')
('Epoch:', '0007', 'training time (minutes)=', '1.3324', 'loss=', '225.068549129')
('Epoch:', '0008', 'training time (minutes)=', '1.3251', 'loss=', '221.883856310')
('Epoch:', '0009', 'training time (minutes)=', '1.3383', 'loss=', '224.151034001')
('Epoch:', '0010', 'training time (minutes)=', '1.3345', 'loss=', '223.320398385')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/8_cabinet_2048_4096pts_emd/models.ckpt-10 is not in all_model_checkpoint_paths. Manually adding it.


...

INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/8_cabinet_2048_4096pts_emd/models.ckpt-720 is not in all_model_checkpoint_paths. Manually adding it.


('Epoch:', '0721', 'training time (minutes)=', '1.3272', 'loss=', '144.730486734')
('Epoch:', '0722', 'training time (minutes)=', '1.3292', 'loss=', '144.770867266')
('Epoch:', '0723', 'training time (minutes)=', '1.3332', 'loss=', '144.232172067')
('Epoch:', '0724', 'training time (minutes)=', '1.3314', 'loss=', '144.818567984')
('Epoch:', '0725', 'training time (minutes)=', '1.3280', 'loss=', '144.134215110')
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/8_cabinet_2048_4096pts_emd/models.ckpt-725 is not in all_model_checkpoint_paths. Manually adding it.


INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/8_cabinet_2048_4096pts_emd/models.ckpt-725 is not in all_model_checkpoint_paths. Manually adding it.


('Epoch:', '0726', 'training time (minutes)=', '1.3735', 'loss=', '143.927626474')
('Epoch:', '0727', 'training time (minutes)=', '1.3400', 'loss=', '143.639113072')


KeyboardInterrupt: 